<div style="font-weight: bold;font-size:40px">Introduction</div>

><div style="background-color: #FBFFEE;">
><ul style="font-size:20px;">
 Hello everyone!
>    
>    I believe that the best approach to learn is to learn what you're passionate about.
>So, I'm passionate about healthy food 🥑 Let's learn together and investigate avocado prices!
>We will discover this question with the use of EDA (the second step) and ML methods (the third step).
>    
>Many thanks to the сreator of this dataset,
><br>If you like this project, please, support me - UPvote!😃

<center><img src="https://i.pinimg.com/564x/cf/c4/f1/cfc4f1cfd6d9af866b8cd3ace353b6d5.jpg" width=300></center>


</ul>
</div>


<div style="font-weight: bold;font-size:30px">Data columns description & libraries</div>

><div style="background-color: #FBFFEE;">
><ul style="font-size:18px;">
<li> Date - The date of the observation</li>
<li> AveragePrice - the average price of a single avocado</li>
<li> type - conventional or organic</li>
<li> year - the year</li>
<li> Region - the city or region of the observation</li>
<li> Total Volume - Total number of avocados sold</li>
<li> 4046 - Total number of avocados with PLU 4046 sold</li>
<li> 4225 - Total number of avocados with PLU 4225 sold</li>
<li> 4770 - Total number of avocados with PLU 4770 sold</li>
</ul>
</div>

<br><div style="font-weight: bold;font-size:30px">Table of Contents</div>

* [Step 1: Examining Data](#section-two)
    - [Basic information](#sub-21)
    - [Data preproccessing](#sub-22)
    - [Making additional columns](#sub-23)

* [Step 2: EDA](#section-three)
    - [2.1 Research of daily, monthly and per year average price](#sub-31)
    - [2.2 Examining features](#sub-32)
    - [2.3 The correlation matrix](#sub-33)
    
* [Step 3: Regression](#section-four)
    - [2.1 Research of daily, monthly and per year average price](#sub-31)
    - [2.2 Examining features](#sub-32)
    - [2.3 The correlation matrix](#sub-33)
* [Overall Conclusion](#section-end)
    

In [ ]:
#Importing Requierd Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

# for Interactive Shells
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


#removing warnings
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report 

# to depict tree_prediction
! pip install pydotplus
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz
from IPython.display import Image

#secrets
from kaggle_secrets import UserSecretsClient
secret_label = "notebook_secret"
secret_value = UserSecretsClient().get_secret(secret_label)

<a id="section-two"></a>
<div style="font-weight: bold;font-size:30px">Step 1: Examining Data</div>

<a id="sub-21"></a>
<div style="font-weight: bold;font-size:20px">1.1 Basic information</div>

In [ ]:
#loading data
try:
    df = pd.read_csv('/kaggle/input/avocado-prices/avocado.csv')
except:
    df = pd.read_csv('avocado.csv')
    
#making a function for examining data
def data_research(data, data_name='data'):
    #basic
    print(f'Examining "{data_name}"')
    display(data.head())
    display(data.info())
    display(data.describe( include='all'))
    display(data.columns)
    
    #duplicates
    duplicates = data.duplicated().sum()
    if duplicates > 0:
        print('There are no duplicated entries.')
    else:
        print(f'There are {duplicates} duplicates.')
        
    #missing
    data_missing = pd.DataFrame(round(data.isnull().sum() / data.shape[0] * 100, 2))
    if data_missing[0].sum() > 0:
        print(f'Missing values in the "{data_name}":')
        data_missing.plot(kind='bar')
    else:
        print(f'There are no missing values in "{data_name}".')
    
    #unique values
    for i in data.columns:
        if data[i].dtype == 'object' or data[i].dtype == 'str':
            print(data[i].unique())
    
data_research(df, data_name='Avocado dataset')

><div style="background-color: #FBFFEE;">
><div style="font-weight: bold;font-size:20px">Observations</div>
><ul style="font-size:18px;">
><center><img src="https://i.pinimg.com/originals/c2/1e/cb/c21ecb47122ea4d8eeab4d9ae968cc36.jpg", width=650></center>
We've examined data:
<br>- There are 18249 entries and 14 columns, one column is an index, so it is needed to drop it;
<br>- No missing or duplicated values and errors (at the first glance), all unique values are correct and don't repeat;
<br>- There are some columns, which datatypes are should be changed.<br>
At the next step we'll study and preprocess outliers, rename column names and change datatypes.
></ul>
></div>

<a id="sub-22"></a>
<div style="font-weight: bold;font-size:20px">1.2 Data preproccessing</div>

<div style="font-weight: bold;font-size:16px">1.2.1 Renaming columns and drop the unecessary one</div>

In [ ]:
df.columns = df.columns.str.lower()
df.drop('unnamed: 0', axis=1, inplace=True)
df.columns

<div style="font-weight: bold;font-size:16px">1.2.2 Changing data types</div>

In [ ]:
for i in df.columns:
    if i == 'date':
        df[i] = df[i].astype('datetime64[ns]')
    elif df[i].dtype == 'object':
        df[i] = df[i].astype('category')

numeric_columns = ['averageprice', 'total volume',
                   '4046', '4225', '4770',
                   'total bags', 'small bags',
                   'large bags', 'xlarge bags']
categorical_columns = ['region', 'type']
data_columns = ['data', 'year']
display(df.info())

<div style="font-weight: bold;font-size:16px">1.2.3 Getting known with the kind of distrubutions</div>

In [ ]:
# distributions
def dist_custom(dataset, columns_list, rows, cols, suptitle):
    fig, axs = plt.subplots(rows, cols,figsize=(16,16))
    fig.suptitle(suptitle,y=0.92, size=16)
    axs = axs.flatten()
    for i, data in enumerate(columns_list):
        sns.distplot(dataset[data], ax=axs[i])
        axs[i].set_title(data + ', skewness is '+str(round(dataset[data].skew(axis = 0, skipna = True),2)))
        
dist_custom(dataset=df, columns_list=numeric_columns, rows=3, cols=3, suptitle='Distibution for each variable')

<div style="font-size:16px">
<b>Conclusion:</b> all features are skewed to the left, there is no Normal Distribution.

Let's examine outliers with the use of boxplots.</div>


In [ ]:
# outliers
def boxplots_custom(dataset, columns_list, rows, cols, suptitle):
    fig, axs = plt.subplots(rows, cols, sharey=True, figsize=(16,12))
    fig.suptitle(suptitle,y=0.93, size=16)
    axs = axs.flatten()
    for i, data in enumerate(columns_list):
        if i % 3 == 0:
            axs[i].set_ylabel('The number of entries')
        sns.boxplot( data=dataset[data], orient='h', ax=axs[i])
        axs[i].set_title(data)
        
boxplots_custom(dataset=df, columns_list=numeric_columns, rows=3, cols=3, suptitle='Boxplots before deleting outliers')

 <div style="font-size:16px">For Skewed distributions we'll use Inter-Quartile Range (IQR) proximity rule.</div>

In [ ]:
# deleting outliers
Q1 = df[numeric_columns].quantile(0.25)
Q3 = df[numeric_columns].quantile(0.75)
IQR = Q3 - Q1
print('Here we will get IQR for each column\n',IQR)

df_filtered = df[~((df[numeric_columns] < (Q1 - 1.5 * IQR)) |(df[numeric_columns] > (Q3 + 1.5 * IQR))).any(axis=1)]
display(df_filtered.shape)

In [ ]:
boxplots_custom(dataset=df_filtered, 
                columns_list=numeric_columns, 
                rows=3, cols=3, suptitle='Boxplots after deleting outliers')

><div style="background-color: #FBFFEE;">
><div style="font-weight: bold;font-size:20px">Observations</div>
><ul style="font-size:18px;">
>
>Looks much better now! But nevertheless we see some outliers even in filtered data. It will be great to scale the features at the step of building model.
We'll use filtered data further
> 
></ul>
></div>

<a id="sub-23"></a>
<div style="font-weight: bold;font-size:20px">1.3 Making additional columns to filtered data</div>

We already have information about day and year of price scanning, nor let's add additional columns for the further cohort research:
- week;
- month.

In [ ]:
df_filtered['month'] = df_filtered['date'].astype('datetime64[M]')
df_filtered['week'] = df_filtered['date'].astype('datetime64[W]')
bins = [0.48, 1.52, 1.78, 1.9, 2.49]
labels = ["low","mean","high",'expensive']
df_filtered['price_types'] = pd.cut(df['averageprice'], bins=bins, labels=labels)
df_filtered.head()

><div style="background-color: #FBFFEE;">
><div style="font-weight: bold;font-size:20px">Conclusion:</div>
><ul style="font-size:18px;">
> At this stage:
><li> examined the data;</li>
><li> renamed column name and drop the unecessary column 'Unnamed: 0';</li>
><li> changed dtypes;</li>
><li> observed the kind of disributions and calculated the skewness value for each numeric column;</li>
><li> plotted boxcharts before and after removing outliers;</li>
><li> added new columns for the further research.</li>
>Now we're going to make Exploratory Data Analysis➡
></ul>
></div>

<a id="section-three"></a>
<div style="font-weight: bold;font-size:30px">Step 2: EDA</div>

<a id="sub-31"></a>
><div style="font-weight: bold;font-size:20px">2.1 Research of daily, monthly and per year average price</div>
><div style="font-size:16px">
Let's create new dataset with avarage avocado price per day, month and year. After that we'll create lineplots with the use of plotly.express and analyze it</div>

In [ ]:
# Initialize figure with subplots
fig = make_subplots(
    rows=2, cols=2, subplot_titles=("Daily average prices", "Weekly average prices",
                                    "Monthly average prices", "Average prices per years")
                                        )

datasets = []
for i in ['date','week','month','year']:
    datasets.append(round(df_filtered.groupby(i)['averageprice'].mean().reset_index(),3))
r, c = 1,1 #rows, cols
    
for i, d in enumerate(datasets):
    # Add traces
    fig.add_trace(go.Scatter(x=d.iloc[:,0], y=d['averageprice']), row=r, col=c)
    fig.update_xaxes(title_text='Per ' + d.iloc[:,0].name,row=r, col=c)
    fig.update_yaxes(title_text="The sum of avarage price", row=r, col=c)
    if i == 1:
        r, c = 2, 1
    else:
        c += 1
    
# Update title and height
fig.update_layout(showlegend=False, title_text="Customizing Subplot Axes", height=700)

fig.show()

In [ ]:
# Initialize figure with subplots
fig = make_subplots(
    rows=1, cols=2, subplot_titles=("Daily average prices", 
                                    "Monthly average prices"))

datasets = []
for i in ['date','month']:
    for j in ['conventional', 'organic']:
        datasets.append(round(df_filtered.query('type == @j').groupby(i)['averageprice'].mean().reset_index(),3))
r, c = 1,1
legend_ = ['conventional', 'organic']
for i, d in enumerate(datasets):
    # Add traces
    fig.add_trace(go.Scatter(x=d.iloc[:,0], y=d['averageprice'], name=legend_[i%2]+' per ' + d.iloc[:,0].name), row=r, col=c)
    fig.update_xaxes(title_text='Per ' + d.iloc[:,0].name,row=r, col=c)
    fig.update_yaxes(title_text="The sum of avarage price", row=r, col=c)
    if i == 1:
        c += 1
    
# Update title and height
fig.update_layout(showlegend=True, title_text="Daily and monthly avarage prices for conventional and organic types", height=700)

fig.show()

><div style="background-color: #FBFFEE;">
><div style="font-weight: bold;font-size:20px">Observations:</div>
><ul style="font-size:18px;">
 From the lineplots above:
><li>At the daily and monthly average price we see  a decline in price in January 2016-2017 y. We can conclude that in winter avocados are not in demand, or there were a too much avocados in shops; </li>
><li>At the average price per year chart we see a steady decline from 2015 to 2016 y. </li>
><li>  After that there is an increase, the peak is in 2017 year.</li>
></ul>
></div>

<a id="sub-32"></a>
><div style="font-weight: bold;font-size:20px">2.2 Examining features</div>
><div style="font-size:16px"> At this step we'll analyse the categorical and numeric variables</div>

><div style="font-size:16px">2.2.1 Numeric columns</div>
Histograms, scatterplots

In [ ]:
# histograms
param_graphs = df_filtered.hist(numeric_columns, figsize=(16, 10), bins=20)
plt.suptitle("Hists after removing outliers", y=0.96, size=16)
for axis in param_graphs.flatten():
    axis.set_ylabel('frequency')
plt.show()

Distributions  and scatter plot for each variable according to type variable

In [ ]:
columns_for_research = ['averageprice', 'total volume', '4046','4225','4770', 'type']
g = sns.pairplot(data=df_filtered[columns_for_research], hue="type")
g.fig.suptitle("Distributions and scatter plots for each variable depending on type", y=1.01, size=16)
for ax in g.axes.flat: 
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.show()

><div style="font-size:16px">2.2.2 Categorical columns</div>
Barplots

In [ ]:
ten_largest_volume_regions = df_filtered.groupby(['region'])['total volume'].sum().sort_values(ascending=False).reset_index().head(10)
fig = px.bar(ten_largest_volume_regions,x='region', y='total volume', title='Top ten regions with the greatest total volume over the time')
fig.show()

In [ ]:
df_count_type = df_filtered.groupby(['price_types', 'type'])['date'].count().reset_index()

plt.figure(figsize=(10, 5))
sns.countplot(x='price_types', alpha=0.7, hue='type', 
                  data=df_filtered)
plt.legend( bbox_to_anchor=(1.1, 1.1), loc='upper left')
plt.xlabel('Price types'), plt.ylabel('Amount')
plt.title('Relation between type and average price', size=16, y=1.01);

><div style="background-color: #FBFFEE;">
><div style="font-weight: bold;font-size:20px">Observations:</div>
><ul style="font-size:18px;">
 From the lineplots above:
><li>At the daily and monthly average price we see  a decline in price in January 2016-2017 y. We can conclude that in winter avocados are not in demand, or there were a too much avocados in shops; </li>
><li>At the average price per year chart we see a steady decline from 2015 to 2016 y. </li>
><li>  After that there is an increase, the peak is in 2017 year.</li>
></ul>
></div>

<a id="sub-33"></a>
><div style="font-weight: bold;font-size:20px">2.3 The Correlation matrix</div>
><div style="font-size:16px"> Correlation matrix depicts the correlation coefficients between all pairs of features in the data.
>
> We use the Pearson correlation coefficient, which is a measure of the linear association between two variables. It has a value between -1 and 1 where:
>
><li>-1 indicates a perfectly negative linear correlation between two variables</li>
><li>0 indicates no linear correlation between two variables</li>
><li>1 indicates a perfectly positive linear correlation between two variables</li></div>

In [ ]:
# calculating the correlation matrix
corr = df_filtered.corr()
matrix = np.triu(corr)
plt.figure(figsize=(10, 8))
sns.heatmap(corr, vmax=1.0, vmin=-1.0, 
            fmt='.1g', annot=True, mask = matrix)

plt.title('Correlation matrix', size=16)
plt.show()

><div style="background-color: #FBFFEE;">
><div style="font-weight: bold;font-size:20px">BIG observations about all the above:</div>
><ul style="font-size:18px;">
><li>At the daily and monthly average price we see  a decline in price in January 2016-2017 y. We can conclude that in winter avocados are not in demand, or there were a too much avocados in shops; </li>
><li>At the average price per year chart we see a steady decline from 2015 to 2016 y. </li>
><li>  After that there is an increase, the peak is in 2017 year.</li>
></ul>
></div>

<a id="section-end"></a>
<div style="font-weight: bold;font-size:40px">The overall conclusion</div>

><div style="background-color: #FBFFEE;">
><ul style="font-size:20px;">


></ul>
></div>

<a id="section-end"></a>
<div style="font-size:40px" align=center>Work in Progress
<img src="https://i.pinimg.com/564x/9a/42/79/9a4279006e123929b83ad139c42c5da6.jpg" width=50>

</div>

><div style="background-color: #FBFFEE;">
><ul style="font-size:20px;">
 <b>Thank you</b> so much for reading my project. 
 <br>Please, UPvote, if you like it or find it usefull!🍀

></ul>
></div>